# ACE Neural Validation Experiments (In-Silico)

#### Dhuvarakesh Karthikeyan and Jin Seok (Andy) Lee

In [2]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

import os
import re

from acelib.elispot import ELISpot
#from acelib.main import run_ace_golfy

In [3]:
from acelib.sequence_features import AceNeuralEngine
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

ESM2_TOKENIZER = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")
ESM2_MODEL = AutoModelForMaskedLM.from_pretrained("facebook/esm2_t6_8M_UR50D", return_dict=True, output_hidden_states=True)

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
ace_eng = AceNeuralEngine(ESM2_MODEL, ESM2_TOKENIZER, device)
ace_eng.load_weights('../saved_models/trained_model3.pt')
#ace_eng.load_weights('../saved_models/trained_model3.pt')

#ace_eng.embed_sequences(['CASS', 'MASS', 'SASS'])
ace_eng.find_paired_peptides(['peptide_1', 'peptide_2', 'peptide_3'], ['CASS', 'MASS', 'SASS'], sim_fxn='euclidean', threshold=0.46)

[('peptide_1', 'peptide_2'),
 ('peptide_1', 'peptide_3'),
 ('peptide_2', 'peptide_1'),
 ('peptide_2', 'peptide_3'),
 ('peptide_3', 'peptide_1'),
 ('peptide_3', 'peptide_2')]

In [3]:
all_df = pd.read_csv('val_refs/iedb_mmer_all.csv')
pos_df = all_df[all_df['Binding']==1].reset_index(drop=True)
neg_df = all_df[all_df['Binding']==0].reset_index(drop=True)

print(f'Number of positive examples: {len(pos_df)}')
print(f'Number of negative examples: {len(neg_df)}')

Number of positive examples: 2297
Number of negative examples: 4760


In [8]:
class AceSimulatedExperiment:
    """
    Class to simulate an ELISPOT experiment with optimal peptide pools.

    Experiment procedure:
    1. Initialize objecty
    2. Load reference CSV
    3. Get optimized config
    4. Simulate ELISPOT assay
    5. Calculate sensitivity and specificity
    """
    def __init__(self, num_peptides, num_positives, peptides_per_pool, coverage, disallowed_peptides=False, enforced_peptides=True,
                 sliding_window_sample=False, deterministic=False, dispersion_factor=1, num_processes=8, random_seed=70203):
        # Define the pooled ELISPOT parameters
        self.num_peptides = num_peptides
        self.num_positives = num_positives

        self.peptides_per_pool = peptides_per_pool
        self.coverage = coverage

        # Define ACE parameters
        self.disallowed_peptides = disallowed_peptides
        self.enforced_peptides = enforced_peptides
        if self.disallowed_peptides:
            self.enforced_peptides = False
        
        # Define the simulation parameters
        self.sliding_window_sample = sliding_window_sample
        self.deterministic = deterministic
        self.dispersion_factor = dispersion_factor
        self.num_processes = num_processes
        self.random_state = random_seed
    
    def load_reference_csv(self, path):
        """
        Load the reference immunogenicity data from a csv format.
        At minimum this data should have peptide, MHC allele, and 
        whether or not the pMHC complex is immunogenic.

        Expected Columnn Names for the CSV:
            - Epitope: The peptide sequences, all capital letters
            - Allele: The MHC allele: HLA-[A,B,C]*[0-9][0-9]:[0-9][0-9]
            - Binding: 1 if the pMHC was shown to be immunogenic, 0 otherwise
        """
        assert os.path.exists(path), f'File does not exist: {path}'
        data_df = pd.read_csv(path)
        assert 'Epitope' in data_df.columns, f'Epitope column not found in {path}'
        assert 'Binding' in data_df.columns, f'Binding column not found in {path}'
        data_df.sort_values(by=['Epitope'], inplace=True)
        self.data = data_df

    def __run__(self):
        """
        Run one iteration of the simulation.
        """
        # Sample the peptides
        peptide_ids, peptide_sequences, labels, disallowed_peptide_pairs, enforced_peptide_pairs = self.sample_peptides()
        
        # Run ACE to get the optimal configuration
        assay, config_df = self.compute_optimized_config(peptide_ids, peptide_sequences, disallowed_peptide_pairs, enforced_peptide_pairs)
        
        # Simulate the ELISPOT assay
        res_df = self.simulate_spot_counts(assay, config_df, peptide_ids, labels)

        hits = res_df['peptide_id'].values
        additional_pools = len(res_df[res_df['deconvolution_result']=='candidate_hit']['peptide_id'].values)
        total_pools = len(config_df['pool_id'].unique()) + additional_pools

        positive_peptide_ids = [f'peptide_{i}' for i in range(len(peptide_ids)) if labels[i]==1]
        negative_peptide_ids = [f'peptide_{i}' for i in range(len(peptide_ids)) if labels[i]==0]

        # Calculate the sensitivity and specificity
        sensitivity = len(set(hits).intersection(set(positive_peptide_ids)))/len(positive_peptide_ids)
        specificity = len(set(hits).intersection(set(positive_peptide_ids)))/len(hits)

        #print(f'Number of positive peptides: {len(positive_peptide_ids)}')
        #print(f'Number of identified peptides: {len(hits)}')

        res_df['Label'] = [1 if id in positive_peptide_ids else 0 for id in res_df['peptide_id']]
        return res_df, sensitivity, specificity, total_pools
        
    def run(self, num_iterations=1):
        sensitivities = []
        specificities = []
        total_pools = []
        for _ in range(num_iterations):
            res_df, sensitivity, specificity, pools = self.__run__()
            sensitivities.append(sensitivity)
            specificities.append(specificity)
            total_pools.append(pools)
        if num_iterations == 1:
            print(f'Sensitivity: {sensitivity}')
            print(f'Specificity: {specificity}')
            return res_df, total_pools
        return total_pools

    def sample_peptides(self):
        """
        Sample the peptide sequences and their immunogenicity status
        from the reference data.

        NOTE: We do this instead of randomly initializing peptide sequences
        because the ACE Neural Engine was trained on real sequence data. Thus,
        to make meaningful disallowed peptide pairings we utilize a reference
        dataset.
        """
        # Sample the peptides
        pos_df = self.data[self.data['Binding']==1].reset_index(drop=True)
        neg_df = self.data[self.data['Binding']==0].reset_index(drop=True)

        if self.sliding_window_sample:
            pos_idx = np.random.randint(0, len(pos_df)-self.num_positives)
            neg_idx = np.random.randint(0, len(neg_df)-(self.num_peptides))
            # Pick peptides that are close to each other
            positive_peptides = self.data[self.data['Binding']==1].iloc[pos_idx:pos_idx+self.num_positives]
            negative_peptides = self.data[self.data['Binding']==0].iloc[neg_idx:(neg_idx+self.num_peptides-self.num_positives)]
        else:
            positive_peptides = self.data[self.data['Binding']==1].sample(self.num_positives)
            negative_peptides = self.data[self.data['Binding']==0].sample(self.num_peptides-self.num_positives)
        
        # Combine the peptides and shuffle them
        peptides = pd.concat([positive_peptides, negative_peptides]).sample(frac=1).reset_index(drop=True)
        
        #############################
        ### Temporary for testing ###
        #############################
        # to be replaced by bonafide sequence similarity
        positives = peptides[peptides['Binding']==1]
        disallowed_peptide_pairs = []
        enforced_peptide_pairs = []
        if self.disallowed_peptides:
            for idx, _ in positives.iterrows():
                for idx2, _ in positives.iterrows():
                    if idx != idx2:
                        disallowed_peptide_pairs.append((f'peptide_{idx}', f'peptide_{idx2}'))
        if self.enforced_peptides:
            enforced_peptide_pairs = []
            for idx, _ in positives.iterrows():
                for idx2, _ in positives.iterrows():
                    if idx != idx2:
                        enforced_peptide_pairs.append((f'peptide_{idx}', f'peptide_{idx2}'))
        #############################
        #############################
    
        peptide_ids = [f'peptide_{idx}' for idx in peptides.index.values]
        peptide_sequences = peptides['Epitope'].values
        labels = peptides['Binding'].values
        return peptide_ids, peptide_sequences, labels, disallowed_peptide_pairs, enforced_peptide_pairs

    def compute_optimized_config(self, peptide_ids, peptide_sequences, disallowed_peptide_pairs, enforced_peptide_pairs):
        """
        Run ACE to generate the optimal peptide pools.
        """
        peptide_df = pd.DataFrame({'peptide_id': peptide_ids, 'peptide_sequence': peptide_sequences})

        # Create the ELISPOT assay object
        assay = ELISpot(
            num_peptides_per_pool=self.peptides_per_pool,
            num_coverage=self.coverage,
            num_processes=10,
            df_peptides=peptide_df
        )

        # Generate the configuration using the optimization algorithm
        generation_status, config_df = assay.generate_configuration(disallowed_peptide_pairs=disallowed_peptide_pairs, enforced_peptide_pairs=enforced_peptide_pairs, random_seed=self.random_state)
        try:
            assert len(config_df) > 0
        except:
            raise AssertionError(f'No valid configurations found for the assay parameters. Please try again with different parameters.')
        return assay, config_df
    
    @staticmethod
    def sample_spot_counts(mean: float, dispersion_factor, num_samples):
        """
        Sample the number of spots for a peptide pool using a negative binomial distribution.

        Derivation of NegBinom Parameters:

            Let X be the number RV of spots sampled for a peptide pool.
            Let p be the probability of sampling a spot for a peptide in the pool.
            Let r be the number of successes (spots) we want to sample.
            Let k be the number of failures (non-spots) we want to sample.

            Then, the probability mass function for X is given by:

            P(X=k) = (k+r-1)C(k) * p^r * (1-p)^k
            
            where (k+r-1)C(k) is the binomial coefficient.

            The mean and variance of X are given by:

            mean = r(1-p)/p
            var = r(1-p)/p^2

            We can solve for p and r in terms of the mean and variance:

            p = mean / var
            r = mean^2 / (var - mean)
        """
        if dispersion_factor < 1:
            raise ValueError("dispersion_factor must be greater than or equal to 1")
        elif dispersion_factor == 1:
            return np.random.poisson(mean, num_samples)
        else:    
            variance = mean*dispersion_factor
            p = mean/variance
            r = mean**2/(variance-mean)
            return np.random.negative_binomial(r, p, num_samples)

    def simulate_spot_counts(self, assay, config_df, peptide_ids, labels):
        """
        Simulate the ELISPOT assay using the optimal peptide pools configuration.
        Determine how spots are sampled for each peptide pool. 

        Assumptions:
            1. The number of spots sampled for each peptide pool follows a negative binomial distribution.
            2. The number of spots sampled for each peptide pool is independent of the other peptide pools.
            3. Immunogenic and non-immunogenic peptides have the same shape parameters for the negative binomial distribution, 
            but different mean parameters. This is because we assume the experimental error/noise is the same for both whereas
            true immunogenic peptides will likely have more spots than non-immunogenic peptides.
            4. We assume that having multiple immunogenic peptides in a pool addiditively increases the number of spots sampled


        Parameters:
            config_df: The optimal peptide pools configuration dataframe
            peptide_ids: The peptide ids for each peptide sequence
            labels: The immunogenicity labels for each peptide sequence

        Returns:
            hit_pool_list: A list of the pool ids that were determined to be hits
        """
        label_dict = {peptide_id:label for peptide_id, label in zip(peptide_ids, labels)}
        hit_pool_list = []
        # In the deterministic case we just use the labels to get 0/1 spot counts
        if self.deterministic:
            # Check to see if that pool has an immunogenic peptide in it
            for pool_id in config_df['pool_id'].unique():
                pool_df = config_df[config_df['pool_id']==pool_id]
                for peptide_id in pool_df['peptide_id']:
                    if label_dict[peptide_id] == 1:
                        hit_pool_list.append(pool_id)
                        break

        # Simulate according to a probabilistic model
        else:
            immunogenic_mean = 100
            non_immunogenic_mean = 10
            
            pool_spot_counts = {i:0 for i in config_df['pool_id'].unique()}
            for pool_id in config_df['pool_id'].unique():
                pool_df = config_df[config_df['pool_id']==pool_id]
                for peptide_id in pool_df['peptide_id']:
                    if label_dict[peptide_id] == 1:
                        pool_spot_counts[pool_id] += AceSimulatedExperiment.sample_spot_counts(immunogenic_mean, self.dispersion_factor, num_samples=1)[0]
                    else:
                        pool_spot_counts[pool_id] += AceSimulatedExperiment.sample_spot_counts(non_immunogenic_mean, self.dispersion_factor, num_samples=1)[0]
                    # Check to see if we have enough spots to call it a hit
                    if pool_spot_counts[pool_id] >= immunogenic_mean:
                            hit_pool_list.append(pool_id)
                            break    
        
        res_df = assay.identify_hit_peptides(hit_pool_list, config_df)
        return res_df


In [9]:
simulation = AceSimulatedExperiment(
                                    num_peptides=25,
                                    num_positives=3,
                                    peptides_per_pool=5,
                                    coverage=3,
                                    disallowed_peptides=False,
                                    enforced_peptides=True,
                                    sliding_window_sample=False,
                                    deterministic=True,
                                    dispersion_factor=2,
                                    num_processes=8
                                )
simulation.load_reference_csv('val_refs/iedb_mmer_all.csv')
total_pools = simulation.run(100)

min(total_pools), max(total_pools), np.median(total_pools), np.mean(total_pools), np.std(total_pools)
#sensitivities, specificities = simulation.run(1)
#min(sensitivities), max(sensitivities), np.median(sensitivities), np.mean(sensitivities), np.std(sensitivities)


TypeError: generate_configuration() got an unexpected keyword argument 'enforced_peptide_pairs'

In [46]:
min(total_pools), max(total_pools), np.median(total_pools), np.mean(total_pools), np.std(total_pools)

(15, 22, 16.0, 15.93, 1.4370455803487932)

In [44]:
simulation = AceSimulatedExperiment(
                                    num_peptides=25,
                                    num_positives=3,
                                    peptides_per_pool=5,
                                    coverage=3,
                                    disallowed_peptides=False,
                                    enforced_peptides=False,
                                    sliding_window_sample=False,
                                    deterministic=True,
                                    dispersion_factor=2,
                                    num_processes=8
                                )
simulation.load_reference_csv('val_refs/iedb_mmer_all.csv')
total_pools = simulation.run(100)
#sensitivities, specificities = simulation.run(100)

min(total_pools), max(total_pools), np.median(total_pools), np.mean(total_pools), np.std(total_pools)


2023-07-03 22:48:14 INFO     CP solver started.
2023-07-03 22:48:14 INFO     CP solver finished.
2023-07-03 22:48:14 INFO     An optimal feasible solution was found.
2023-07-03 22:48:14 INFO     CP solver started.
2023-07-03 22:48:15 INFO     CP solver finished.
2023-07-03 22:48:15 INFO     An optimal feasible solution was found.
2023-07-03 22:48:15 INFO     CP solver started.
2023-07-03 22:48:15 INFO     CP solver finished.
2023-07-03 22:48:15 INFO     An optimal feasible solution was found.
2023-07-03 22:48:15 INFO     CP solver started.
2023-07-03 22:48:16 INFO     CP solver finished.
2023-07-03 22:48:16 INFO     An optimal feasible solution was found.
2023-07-03 22:48:16 INFO     CP solver started.
2023-07-03 22:48:16 INFO     CP solver finished.
2023-07-03 22:48:16 INFO     An optimal feasible solution was found.
2023-07-03 22:48:16 INFO     CP solver started.
2023-07-03 22:48:16 INFO     CP solver finished.
2023-07-03 22:48:16 INFO     An optimal feasible solution was found.
2023

(15, 22, 16.0, 16.07, 1.5764199947983406)

### Modeling the ELISPOT False Discovery Rate

#### Without Sequence Features

In [5]:
### Sample without replacement
np.random.randint(0, 100, 10, dtype=np.int32, endpoint=True)

TypeError: randint() got an unexpected keyword argument 'with_replacement'

#### With Sequence Features